In [ ]:
import dalmatian as dm
import pandas as pd

In [ ]:
ccle_ws = dm.WorkspaceManager("fccredits-silver-tan-7621/CCLE_v2")
ccle_samples = ccle_ws.get_samples()
ccle_wgs = ccle_samples[~ccle_samples.hg38_wgs_bam.isna()][["hg38_wgs_bam"]]

In [ ]:
ccle_wgs['vcf'] = ""
ccle_wgs

In [ ]:
ccle_wgs["vcf_index"] = ""

In [ ]:
wgs_ws = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WGS_CN")
wgs_samples = wgs_ws.get_samples()

In [ ]:
from genepy.utils import helper as h

cmds = []
for i in ccle_wgs.index:
    bam = ccle_wgs.loc[i, "hg38_wgs_bam"]
    cdsid = bam.split("/")[-1][:10]
    vcf = wgs_samples.loc[cdsid, "cnn_filtered_vcf"]
    ccle_wgs.loc[i, "vcf"] = "gs://depmap-wgs-vcf/" + vcf.split("/")[-1]
    vcf_idx = wgs_samples.loc[cdsid, "cnn_filtered_vcf_index"]
    ccle_wgs.loc[i, "vcf_index"] = "gs://depmap-wgs-vcf/" + vcf_idx.split("/")[-1]
    cmds.append("gsutil -u broad-firecloud-ccle cp " + vcf + " gs://depmap-wgs-vcf")
    cmds.append("gsutil -u broad-firecloud-ccle cp " + vcf_idx + " gs://depmap-wgs-vcf")

In [ ]:
cmds

In [ ]:
ccle_wgs

In [ ]:
h.parrun(cmds, cores=8)

In [ ]:
ccle_wgs.to_csv("wgs_vcf.csv")